In [1]:
import os
import matplotlib
import matplotlib.colors
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
matplotlib.use("agg")

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Data pickle path
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Plot output path
plot_output_root = "C:\\Users\\dosre\\dev\\thesis-data\\plots"
os.makedirs(plot_output_root, exist_ok=True)
plots_characterization = os.path.join(plot_output_root, "plots_characterization")
os.makedirs(plots_characterization, exist_ok=True)

In [3]:
# Load data
extended_data = pd.read_pickle(extended_data_path)
median_data = pd.read_pickle(median_data_path)
conv_traj_data = extended_data[
    (extended_data["solver"] == "FP FP16") |
    (extended_data["solver"] == "FP FP32") |
    (extended_data["solver"] == "FP FP64")
]

In [4]:
relres_diff_data = {
    "FP FP16": [],
    "FP FP32": []
}

first_deriv_data = {
    "FP FP16": [],
    "FP FP32": []
}

for setup in ["unprecond"]:
    for matrix in df_sel_setup(conv_traj_data, setup)["matrix"].unique():
        for restart_param in df_sel_setup_matrix(conv_traj_data, setup, matrix)["restart_param"].unique():

            sub_data = df_sel_setup_matrix_restart(
                conv_traj_data, setup, matrix, restart_param
            )

            for exp_iter in sub_data["experiment_iter"].unique():

                for solver in ["FP FP16", "FP FP32"]:

                    lowfp_data = sub_data[
                        (sub_data["solver"] == solver) &
                        (sub_data["experiment_iter"] == exp_iter)
                    ]

                    # Calculate first derivative
                    if lowfp_data.size != 0:
                        lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                        fm1 = lowfp_data_row[:(lowfp_data_row.size-2)]
                        f1 = lowfp_data_row[2:]
                        deriv_to_add = np.hstack([
                            (lowfp_data_row[1]-lowfp_data_row[0]),
                            0.5*(f1-fm1),
                            (lowfp_data_row[-1]-lowfp_data_row[-2])
                        ])
                        if 15001-deriv_to_add.size > 0:
                            deriv_to_add = np.hstack(
                                (deriv_to_add,
                                    np.full(15001-deriv_to_add.size, np.nan))
                            )

                        first_deriv_data[solver].append(deriv_to_add)
                        

                    # Calculate relres diff between lowfp and fp64
                    fp64_data = sub_data[
                        (sub_data["solver"] == "FP FP64") &
                        (sub_data["experiment_iter"] == exp_iter)
                    ]
                    if lowfp_data.size != 0:
                        kwargs = {
                            "color": SOLVER_CLR_DICT[solver],
                            "alpha": 0.005,
                            "linestyle": None,
                            "marker": "."
                        }
                        lowfp_data_row = lowfp_data.iloc[0]["inner_relres"]
                        control_data_row = fp64_data.iloc[0]["inner_relres"]
                        if control_data_row.size < lowfp_data_row.size :
                            lowfp_data_row = lowfp_data_row[:control_data_row.size ]
                        elif control_data_row.size > lowfp_data_row.size:
                            control_data_row = control_data_row[:lowfp_data_row.size]
                        
                        diff_data_to_add = lowfp_data_row-control_data_row
                        diff_data_to_add /= control_data_row
                        diff_data_to_add = np.append(
                            diff_data_to_add,
                            np.full(15001-diff_data_to_add.size, np.nan)
                        )
                        
                        diff_data_to_add[diff_data_to_add < 1e-12] = 1e-12
                        
                        relres_diff_data[solver].append(diff_data_to_add)

relres_diff_data["FP FP16"] = np.vstack(relres_diff_data["FP FP16"])
relres_diff_data["FP FP32"] = np.vstack(relres_diff_data["FP FP32"])

diff_max_val = np.nanmax(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))
diff_min_val = np.nanmin(np.hstack((relres_diff_data["FP FP16"], relres_diff_data["FP FP32"])))

first_deriv_data["FP FP16"] = np.vstack(first_deriv_data["FP FP16"])
first_deriv_data["FP FP32"] = np.vstack(first_deriv_data["FP FP32"])

deriv_max_val = np.nanmax(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"])))
deriv_min_val = np.nanmin(np.hstack((first_deriv_data["FP FP16"], first_deriv_data["FP FP32"])))

##### 7.1 Wasted Work Characterization Relres Diff Plots

In [5]:
for solver in ["FP FP16", "FP FP32"]:

    fig, axs = plt.subplots(1, 3, figsize=(6.4, 6.4), width_ratios=[0.49, 0.49, 0.02])
    ax1, ax2, ax3 = axs

    iter_split = 200
    max_iters = relres_diff_data[solver].shape[1]
    indices_iters = np.arange(0, max_iters, 1)
    indices_exp = np.arange(0, relres_diff_data[solver].shape[0], 1)
    X, Y = np.meshgrid(indices_iters, indices_exp)

    ax1.pcolormesh(
        X[:, :iter_split], Y[:, :iter_split],
        relres_diff_data[solver][:, :iter_split],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
    )
    ax1.set_xticks(np.arange(0, iter_split+1, 25))
    ax1.set_ylabel("Linear Solve Experiment")

    ax_im = ax2.pcolormesh(
        X[:, iter_split:], Y[:, iter_split:],
        relres_diff_data[solver][:, iter_split:],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.LogNorm(vmin=diff_min_val, vmax=diff_max_val)
    )
    ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))

    for ax in [ax1, ax2]:
        ax.set_yticks([])
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_xlabel("Inner Iterations")
        ax.grid()
        ax.set_axisbelow(True)

    fig.suptitle(f"{solver.replace("FP ", "")} Rel. Res. Frac. Error from FP64")
    fig.colorbar(ax_im, cax=ax3)
    fig.tight_layout()

    plt.savefig(
        os.path.join(plots_characterization, f"wasted_work_{solver.replace("FP ", "")}.png"),
        bbox_inches='tight'
    )
    plt.close()

##### 7.1 Stagnation Characterization

In [6]:
for solver in ["FP FP16", "FP FP32"]:

    fig, axs = plt.subplots(1, 3, figsize=(6.4, 6.4), width_ratios=[0.49, 0.49, 0.02])
    ax1, ax2, ax3 = axs

    iter_split = 200
    max_iters = relres_diff_data[solver].shape[1]
    indices_iters = np.arange(0, max_iters, 1)
    indices_exp = np.arange(0, relres_diff_data[solver].shape[0], 1)
    X, Y = np.meshgrid(indices_iters, indices_exp)

    ax1.pcolormesh(
        X[:, :iter_split], Y[:, :iter_split],
        first_deriv_data[solver][:, :iter_split],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-10)
    )
    ax1.set_xticks(np.arange(0, iter_split+1, 25))
    ax1.set_ylabel("Linear Solve Experiment")

    ax_im = ax2.pcolormesh(
        X[:, iter_split:], Y[:, iter_split:],
        first_deriv_data[solver][:, iter_split:],
        cmap="RdYlGn_r",
        norm=matplotlib.colors.SymLogNorm(vmin=deriv_min_val, vmax=deriv_max_val, linthresh=1e-10)
    )
    ax2.set_xticks([iter_split] + list(np.arange(2500*int(iter_split/2500+1), max_iters+1, 2500)))

    for ax in [ax1, ax2]:
        ax.set_yticks([])
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_xlabel("Inner Iterations")
        ax.grid()
        ax.set_axisbelow(True)

    fig.suptitle(f"{solver.replace("FP ", "")} First Derivative of Rel. Res.")
    fig.colorbar(ax_im, cax=ax3)
    fig.tight_layout()

    plt.savefig(
        os.path.join(plots_characterization, f"stagnation_{solver.replace("FP ", "")}.png"),
        bbox_inches='tight'
    )
    plt.close()

##### 7.2 Forgetful Mechanism Characterization

In [7]:
setup_data = median_data[median_data["setup" ] == "unprecond"]
for solver in ["FP FP16", "FP FP32"]:
    fig, ax = plt.subplots(figsize=(6.4, 4))

    sub_data = setup_data[setup_data["solver"] == solver]
    sub_data = sub_data[
        ["setup", "matrix", "restart_param", "med_inner_mean_rel_res_frac_err_fp64"]
    ]
    plot_med_y = []
    plot_iqr_25_y = []
    plot_iqr_75_y = []
    for restart_param in RESTART_PARAMS:
        plot_data = sub_data[sub_data["restart_param"] == restart_param]
        plot_med_y.append(np.median(plot_data["med_inner_mean_rel_res_frac_err_fp64"]))
        plot_iqr_25_y.append(np.percentile(plot_data["med_inner_mean_rel_res_frac_err_fp64"], 25))
        plot_iqr_75_y.append(np.percentile(plot_data["med_inner_mean_rel_res_frac_err_fp64"], 75))
    plot_yerr_bot = [plot_med_y[i]-plot_iqr_25_y[i] for i in range(len(RESTART_PARAMS))]
    plot_yerr_top = [plot_iqr_75_y[i]-plot_med_y[i] for i in range(len(RESTART_PARAMS))]
    ax.plot(
        RESTART_PARAMS,
        plot_med_y,
        ".-",
        color=SOLVER_CLR_DICT[solver]
    )
    # ax.errorbar(
    #     RESTART_PARAMS,
    #     # range(len(RESTART_PARAMS)),
    #     plot_med_y,
    #     yerr=[plot_yerr_bot, plot_yerr_top],
    #     fmt=".-",
    #     capsize=3,
    #     color=SOLVER_CLR_DICT[solver]
    # )
    # ax.set_xticks(range(len(RESTART_PARAMS)))
    # ax.set_xticklabels(RESTART_PARAMS)
    ax.grid()
    ax.set_xlabel("Restart Parameter")
    ax.set_ylabel("Median Experimental Average Rel. Res. Frac. Error")

    fig.suptitle(f"{solver} Experimental Average Rel. Res. Frac. Error from FP64")

    plt.savefig(
        os.path.join(plots_characterization, f"forgetful_mechanisms__{solver.replace("FP ", "")}.png"),
        bbox_inches='tight'
    )
    plt.close()